In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('Amazon.csv')

# Sample the data to speed up computation
# Comment out this line to match with lecture

# df = df.sample(frac=0.1, random_state=10)

df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413840 entries, 0 to 413839
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Product Name  413840 non-null  object 
 1   Brand Name    348669 non-null  object 
 2   Price         407907 non-null  float64
 3   Rating        413840 non-null  int64  
 4   Reviews       413778 non-null  object 
 5   Review Votes  401544 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 18.9+ MB


In [13]:
df.isnull().sum()

Product Name        0
Brand Name      65171
Price            5933
Rating              0
Reviews            62
Review Votes    12296
dtype: int64

In [14]:
# Drop missing values
df.dropna(inplace=True)

# We consider rating as neutral so remove it.
df = df[df['Rating'] != 3]

# Encode 4s and 5s as 1 (rated positively)
# Encode 1s and 2s as 0 (rated poorly)

df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)

df.head(12)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,1
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,1
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,1
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,1
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,1
5,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,1,I already had a phone with problems... I know ...,1.0,0
6,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,The charging port was loose. I got that solder...,0.0,0
7,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,"Phone looks good but wouldn't stay charged, ha...",0.0,0
8,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I originally was using the Samsung S2 Galaxy f...,0.0,1
11,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,This is a great product it came after two days...,0.0,1


In [15]:
# Most ratings are positive

# Mean more than 0.5 indicates that we have more 1's than 0's i.e. Most ratings are positive

df['Positively Rated'].mean()

0.7482686025879323

In [16]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets

x = df['Reviews']

y = df['Positively Rated']

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [17]:
print('x_train first entry:\n\n', x_train.iloc[0])

x_train first entry:

 I bought a BB Black and was deliveried a White BB.Really is not a serious provider...Next time is better to cancel the order.


In [18]:
print(x_train.shape, y_train.shape)

(231207,) (231207,)


# CountVectorizer

In scikit-learn's CountVectorizer, features refer to the individual words or terms in a text corpus that the vectorizer uses to create a numerical representation of the text data. The CountVectorizer is a text preprocessing and feature extraction tool commonly used in natural language processing (NLP) tasks, such as text classification and clustering. It converts a collection of text documents into a matrix of token counts, where each row of the matrix corresponds to a document, and each column corresponds to a unique term or word in the entire corpus.

Here's how it works:

1. **Tokenization**: CountVectorizer first tokenizes the input text data, which means it breaks the text into individual words or terms. These terms are considered features.


2. **Feature Extraction**: For each term, it counts how many times that term appears in each document. These counts are then used to create the feature matrix. Each term becomes a feature, and the count of that term in a document is the value associated with that feature for that document.

By default, scikit-learn's CountVectorizer **does not perform stemming or lemmatization**. It treats words as individual tokens without any linguistic normalization. If you want to apply stemming or lemmatization to your text data, you would typically need to do this as a separate preprocessing step before using CountVectorizer.

Stemming and lemmatization are techniques used in natural language processing to reduce words to their base or root forms. Stemming reduces words to a common stem by removing suffixes, while lemmatization reduces words to their dictionary or lemma form.

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

# Fit the CountVectorizer to the training data

vect = CountVectorizer().fit(x_train)

In [20]:
# Here features are all words 

# The CountVectorizer in scikit-learn does not, by default, produce unique tokens. Instead, it 
# tokenizes text by splitting it into individual words or n-grams and then counts the frequency of 
# each token in the text.

len(vect.get_feature_names())

C:\Users\alasp\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


53216

In [21]:
# View some features or words

vect.get_feature_names()[1000:1100]

['2048those',
 '2048x1536',
 '205',
 '2050',
 '2050mah',
 '206',
 '207',
 '2070',
 '20781',
 '207r',
 '2089853',
 '209',
 '20also',
 '20am',
 '20awg',
 '20e',
 '20fps',
 '20ft',
 '20g',
 '20gb',
 '20hours',
 '20hrs',
 '20ish',
 '20mbps',
 '20mile',
 '20min',
 '20mins',
 '20minute',
 '20minutes',
 '20mis',
 '20mp',
 '20pm',
 '20received',
 '20s',
 '20th',
 '20us',
 '20x',
 '21',
 '210',
 '2100',
 '21000',
 '21004g',
 '2100800',
 '2100con',
 '2100ghz',
 '2100lte',
 '2100mah',
 '2100mhz',
 '2100mhzand',
 '2100mhzsim',
 '2100red',
 '2100s4',
 '2105',
 '2105775197',
 '213',
 '2136',
 '2136432',
 '214',
 '215',
 '2153843082',
 '216',
 '2160',
 '217',
 '218',
 '219',
 '21after',
 '21mbps',
 '21mp',
 '21st',
 '21stwifi',
 '21x',
 '21y',
 '22',
 '220',
 '2200',
 '2200mah',
 '220i',
 '220v',
 '2214446',
 '2215',
 '222',
 '2228',
 '222mb',
 '2236053',
 '225',
 '226',
 '2260',
 '227',
 '228',
 '229',
 '2299803resetting',
 '22gb',
 '22hrs',
 '22nd',
 '23',
 '230',
 '2300',
 '2300mah',
 '230mbps',
 

In [22]:
# transform the documents in the training data to a document-term matrix

# Or we can directly use  x_train_vectorized = vect.fit_transform(x_train) in the begining to

# Learn the vocabulary dictionary and return document-term matrix.

x_train_vectorized = vect.transform(x_train)

x_train_vectorized

<231207x53216 sparse matrix of type '<class 'numpy.int64'>'
	with 6117776 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.linear_model import LogisticRegression

# Train the model
model = LogisticRegression(max_iter=300)
model.fit(x_train_vectorized, y_train)

C:\Users\alasp\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(max_iter=300)

In [24]:
from sklearn.metrics import roc_auc_score

x_test_vectorized = vect.transform(x_test)

pred = model.predict(x_test_vectorized)

print('AUC:', roc_auc_score(y_test, pred))

AUC: 0.929997234136369


In [25]:
# get the feature names as numpy array
feature_names = np.array(vect.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model.coef_[0].argsort()

# Find the 10 smallest and 10 largest coefficients
# The 10 largest coefficients are being indexed using [:-11:-1] 
# so the list returned is in order of largest to smallest

# words with smallest coefs are related negative review 
# words with highest coefs are related to positive review

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

C:\Users\alasp\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Smallest Coefs:
['mony' 'worst' 'false' 'horribly' 'worthless' 'blacklist' 'messing'
 'garbage' 'unsatisfied' 'rubbish']

Largest Coefs: 
['excelente' 'excelent' '4eeeks' 'efficient' 'pleasantly' 'excellent'
 'loving' 'exelente' 'satisfy' 'loves']


In [26]:
# But these reviews are not correctly identified
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[0 0]


Since we are **considering only single words** and **not word sequences**, the model is **unable** to **identify difference** between **'issue'** and **'not issue'.**

This **problem** will be **solved** when we will use **ngram_range** in  **CountVectorizer()**

# n-grams

In scikit-learn's CountVectorizer, the **ngram_range** parameter is used to specify the range of n-grams to be extracted from the text data. N-grams are contiguous sequences of n items (words or characters) in a text document. By setting the ngram_range, you can control the size of the n-grams that are considered when vectorizing the text.

The ngram_range parameter takes a tuple with two values: (min_n, max_n). These values define the minimum and maximum n-gram sizes to consider. For example, ngram_range=(1, 2) indicates that **both unigrams (single words) and bigrams (two-word sequences) should be considered** when creating the feature matrix.

In [55]:
# Fit the CountVectorizer to the training data specifiying a minimum 
# document frequency of 5 and extracting 1-grams and 2-grams

vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(x_train)

x_train_vectorized = vect.transform(x_train)

len(vect.get_feature_names())

198917

Since both unigrams (single words) and bigrams (two-word sequences) are considered here, the **Number of features has increased a lot** but we are getting **improved AUC Score.**

In [52]:
model = LogisticRegression()
model.fit(x_train_vectorized, y_train)

predictions = model.predict(vect.transform(x_test))

print('AUC: ', roc_auc_score(y_test, predictions))

C:\Users\alasp\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


AUC:  0.9601780708435216


In [53]:
# Positive and Negative review words are looking more accurate now

feature_names = np.array(vect.get_feature_names())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['no good' 'not happy' 'not worth' 'junk' 'worst' 'not satisfied'
 'garbage' 'not good' 'terrible' 'defective']

Largest Coefs: 
['excelent' 'excelente' 'not bad' 'excellent' 'exelente' 'perfect'
 'no problems' 'awesome' 'no issues' 'perfecto']


C:\Users\alasp\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [54]:
# These reviews are now correctly identified
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[1 0]
